In [1]:
# Import all packages
import numpy as np
import pandas as pd

In [2]:
## parameters
use_diff = True
percent_thresh = 25 # threshold for % activity remaining
diff_percent_thresh = 50
fold_diff_thresh = 50

In [3]:
## Read in data
df1 = pd.read_csv('Klaeger(2017)_Science_processed.csv')
df2 = pd.read_csv('Huang(2017)_eLife_Fig1source_processed.csv')
df3 = pd.read_csv('Annes(2018)_Kinome500.csv') # Annes(2018)_Kinome.csv

In [4]:
df1 = df1[['Name_official','OTS-167','CC-401']]
df1.rename(mapper={'Name_official': 'target', 'OTS-167': 'OTSSP167'}, axis='columns', inplace=True)

if use_diff:
    df1['OTSSP167'] = df1['CC-401'] / df1['OTSSP167'] >= fold_diff_thresh
    df1.drop('CC-401', axis=1, inplace=True)
else:
    df1.loc[df1['CC-401'] < np.inf, 'CC-401'] = True
    df1.loc[df1['CC-401'] == np.inf, 'CC-401'] = False
    df1.loc[df1['OTSSP167'] < np.inf, 'OTSSP167'] = True
    df1.loc[df1['OTSSP167'] == np.inf, 'OTSSP167'] = False

In [5]:
df2 = df2[['Name_official','OTSSP167 (1 μM) % AR','HTH-01-091 (1 μM) % AR']]
df2.rename(mapper={'Name_official': 'target', 'OTSSP167 (1 μM) % AR': 'OTSSP167', 'HTH-01-091 (1 μM) % AR': 'HTH-01-091'}, axis='columns', inplace=True)

if use_diff:
    df2['OTSSP167'] = df2['HTH-01-091'] - df2['OTSSP167'] >= diff_percent_thresh
    df2.drop('HTH-01-091', axis=1, inplace=True)
else:
    df2.loc[(df2['OTSSP167'] < percent_thresh).values, 'OTSSP167'] = True
    df2.loc[df2['OTSSP167'] >= percent_thresh, 'OTSSP167'] = False
    df2.loc[df2['HTH-01-091'] < percent_thresh, 'HTH-01-091'] = True
    df2.loc[df2['HTH-01-091'] >= percent_thresh, 'HTH-01-091'] = False

In [6]:
df3 = df3[['Entrez Gene Symbol','Percent Control (1081)','Percent Control (1285)']]
df3.rename(mapper={'Entrez Gene Symbol': 'target', 'Percent Control (1081)': 'OTSSP167', 'Percent Control (1285)': 'STF-1285'}, axis='columns', inplace=True)

if use_diff:
    df3['OTSSP167'] = df3['STF-1285'] - df3['OTSSP167'] >= diff_percent_thresh
    df3.drop('STF-1285', axis=1, inplace=True)
else:
    df3.loc[df3['OTSSP167'] < percent_thresh, 'OTSSP167'] = True
    df3.loc[df3['OTSSP167'] >= percent_thresh, 'OTSSP167'] = False
    df3.loc[df3['STF-1285'] < percent_thresh, 'STF-1285'] = True
    df3.loc[df3['STF-1285'] >= percent_thresh, 'STF-1285'] = False

In [7]:
result = pd.merge(df1, df2, how='inner', on=['target', 'OTSSP167'])
result = pd.merge(result, df3, how='inner', on=['target', 'OTSSP167'])
result.sort_values(by='target', inplace=True)
result

,target,OTSSP167
11,AURKA,True
6,CAMKK2,True
5,CDK2,True
4,CDK9,True
15,CLK2,False
10,CSNK1D,False
0,DYRK1A,False
3,GSK3B,True
8,IKBKE,True
14,MAP2K1,True


In [8]:
selected = []

if use_diff:
    selected = list(result.loc[result['OTSSP167'] == True, 'target'].values)
else:
    for target in result['target']:
        if np.array_equal(np.squeeze(result.loc[result['target'] == target, ['OTSSP167', 'CC-401', 'HTH-01-091', 'STF-1285']].values.astype(bool)), np.array([True, False, False, False])):
            selected.append(target)

print('\n'.join(selected))

AURKA
CAMKK2
CDK2
CDK9
GSK3B
IKBKE
MAP2K1
MAP3K11
PAK4
RPS6KA3
RPS6KA5
STK16


In [9]:
selected

['AURKA',
 'CAMKK2',
 'CDK2',
 'CDK9',
 'GSK3B',
 'IKBKE',
 'MAP2K1',
 'MAP3K11',
 'PAK4',
 'RPS6KA3',
 'RPS6KA5',
 'STK16']